In [23]:
import sympy
from sympy import symbols, Matrix, cos, sin, diff, Function
from sympy.physics.mechanics import dynamicsymbols, Lagrangian, Particle, ReferenceFrame, Point
from sympy.physics.mechanics import KanesMethod, inertia, RigidBody
from sympy.abc import t 

# Deriving Equations of Motion using SymPy

In [24]:
# Define symbolic variables
M_w, M_r, m_b, R, I_w, I_r, d, h, I_b, r, g, T = symbols('M_w M_r m_b R I_w I_r d h I_b r g T')

# Define time-dependent symbolic functions
theta = dynamicsymbols('theta')
x_w = dynamicsymbols('x_w')
x = dynamicsymbols('x')

In [25]:
# Position (adapted from MATLAB script)
# The MATLAB script uses a rotation matrix, let's replicate that logic.
# The original MATLAB code: pos = [cos(theta), sin(theta); -sin(theta), cos(theta)] * [x; h + r] + [x_w; R];

rotation_matrix = Matrix([[cos(theta), sin(theta)], [-sin(theta), cos(theta)]])
# rotation_matrix = Matrix([[sin(theta), cos(theta)], [-cos(theta), sin(theta)]])
body_coords = Matrix([[x], [h + r]])

rotated_body_coords = rotation_matrix * body_coords

x_b = rotated_body_coords[0] + x_w
y_b = rotated_body_coords[1] 

print("x_b:", sympy.simplify(x_b))
print("y_b:", sympy.simplify(y_b))

x_b: (h + r)*sin(theta(t)) + x(t)*cos(theta(t)) + x_w(t)
y_b: (h + r)*cos(theta(t)) - x(t)*sin(theta(t))


In [26]:
# Potential Energy
U_b = m_b * g * y_b + R
print("U_b = " ,sympy.simplify(U_b))
U_r = M_r * g * (d * cos(theta)) + R
# U_r = M_r * g * (d * cos(theta))
print("U_r = ",sympy.simplify(U_r))
U_w = R
print("U_w = ",sympy.simplify(U_w))
U = sympy.simplify(U_b + U_r + U_w)

print("Total Potential Energy (U):")
print(U)

U_b =  R + g*m_b*((h + r)*cos(theta(t)) - x(t)*sin(theta(t)))
U_r =  M_r*d*g*cos(theta(t)) + R
U_w =  R
Total Potential Energy (U):
M_r*d*g*cos(theta(t)) + 3*R + g*m_b*((h + r)*cos(theta(t)) - x(t)*sin(theta(t)))


In [27]:
# Kinetic Energy
T_w = sympy.Rational(1, 2) * M_w * x_w.diff(t)**2 + sympy.Rational(1, 2) * I_w * (x_w.diff(t)/R)**2
print("T_w = ",T_w)
T_r = sympy.Rational(1, 2) * (M_r * x_w.diff(t)**2 + I_r * theta.diff(t)**2)
print("T_r = " ,T_r)

# T_b - This is the most complex part, carefully translating the MATLAB expression
# MATLAB: 0.5*( m_b*( (diff(x, t)*cos(theta) - x*sin(theta)*diff(theta, t) + (R+r)*cos(theta) + diff(x_w,t) )^2 ...
# + ( -diff(x, t)*sin(theta) - x*cos(theta)*diff(theta, t) - (h+r)*sin(theta) )^2 ) + I_b*(diff(x, t)/r)^2 );

# term1_Tb = (x.diff(t)*cos(theta) - x*sin(theta)*theta.diff(t) + (R+r)*cos(theta) + x_w.diff(t))**2
# term2_Tb = (x.diff(t)*sin(theta) + x*cos(theta)*theta.diff(t) + (h+r)*sin(theta))**2

# T_b = sympy.Rational(1, 2) * (m_b * (term1_Tb + term2_Tb) + I_b * (x.diff(t)/r)**2)
v_b_x = x_b.diff(t)
v_b_y = y_b.diff(t)
T_b_trans = 0.5*m_b*(v_b_x**2 + v_b_y**2)
T_b_rot = 0.5*I_b*((x.diff(t)/r)**2)
T_b  = T_b_trans+ T_b_rot
print("T_b = ",T_b)
T = sympy.simplify(T_w + T_r + T_b)

print("Total Kinetic Energy (T):")

T_w =  I_w*Derivative(x_w(t), t)**2/(2*R**2) + M_w*Derivative(x_w(t), t)**2/2
T_r =  I_r*Derivative(theta(t), t)**2/2 + M_r*Derivative(x_w(t), t)**2/2
T_b =  0.5*I_b*Derivative(x(t), t)**2/r**2 + 0.5*m_b*((-(h + r)*sin(theta(t))*Derivative(theta(t), t) - x(t)*cos(theta(t))*Derivative(theta(t), t) - sin(theta(t))*Derivative(x(t), t))**2 + ((h + r)*cos(theta(t))*Derivative(theta(t), t) - x(t)*sin(theta(t))*Derivative(theta(t), t) + cos(theta(t))*Derivative(x(t), t) + Derivative(x_w(t), t))**2)
Total Kinetic Energy (T):


In [28]:
# Lagrangian
L = T - U
L = sympy.simplify(L)

print("Lagrangian (L):")
print(L)

Lagrangian (L):
(1.0*I_b*R**2*Derivative(x(t), t)**2 + I_w*r**2*Derivative(x_w(t), t)**2 - 2*R**2*r**2*(M_r*d*g*cos(theta(t)) + 3*R + g*m_b*((h + r)*cos(theta(t)) - x(t)*sin(theta(t)))) + R**2*r**2*(I_r*Derivative(theta(t), t)**2 + M_r*Derivative(x_w(t), t)**2 + M_w*Derivative(x_w(t), t)**2 + 1.0*m_b*(h**2*Derivative(theta(t), t)**2 + 2*h*r*Derivative(theta(t), t)**2 + 2*h*cos(theta(t))*Derivative(theta(t), t)*Derivative(x_w(t), t) + 2*h*Derivative(theta(t), t)*Derivative(x(t), t) + r**2*Derivative(theta(t), t)**2 + 2*r*cos(theta(t))*Derivative(theta(t), t)*Derivative(x_w(t), t) + 2*r*Derivative(theta(t), t)*Derivative(x(t), t) + x(t)**2*Derivative(theta(t), t)**2 - 2*x(t)*sin(theta(t))*Derivative(theta(t), t)*Derivative(x_w(t), t) + 2*cos(theta(t))*Derivative(x(t), t)*Derivative(x_w(t), t) + Derivative(x(t), t)**2 + Derivative(x_w(t), t)**2)))/(2*R**2*r**2)


In [29]:
# Generalized coordinates
q = [x_w, theta, x]

# Equations of Motion (Euler-Lagrange)
eom = []
for qi in q:
    eq = diff(diff(L, qi.diff(t)), t) - diff(L, qi)
    eom.append(sympy.simplify(eq))

print("Equations of Motion:")
for i, eq in enumerate(eom):
    print(f"Equation for {q[i]}:")
    # print(eq)
    print(sympy.latex(eq))
    print("\n" + "-"*50 + "\n")

Equations of Motion:
Equation for x_w(t):
\frac{I_{w} \frac{d^{2}}{d t^{2}} x_{w}{\left(t \right)} + \frac{R^{2} \left(2 M_{r} \frac{d^{2}}{d t^{2}} x_{w}{\left(t \right)} + 2 M_{w} \frac{d^{2}}{d t^{2}} x_{w}{\left(t \right)} - 2.0 m_{b} \left(h \sin{\left(\theta{\left(t \right)} \right)} \left(\frac{d}{d t} \theta{\left(t \right)}\right)^{2} - h \cos{\left(\theta{\left(t \right)} \right)} \frac{d^{2}}{d t^{2}} \theta{\left(t \right)} + r \sin{\left(\theta{\left(t \right)} \right)} \left(\frac{d}{d t} \theta{\left(t \right)}\right)^{2} - r \cos{\left(\theta{\left(t \right)} \right)} \frac{d^{2}}{d t^{2}} \theta{\left(t \right)} + x{\left(t \right)} \sin{\left(\theta{\left(t \right)} \right)} \frac{d^{2}}{d t^{2}} \theta{\left(t \right)} + x{\left(t \right)} \cos{\left(\theta{\left(t \right)} \right)} \left(\frac{d}{d t} \theta{\left(t \right)}\right)^{2} + 2 \sin{\left(\theta{\left(t \right)} \right)} \frac{d}{d t} \theta{\left(t \right)} \frac{d}{d t} x{\left(t \right)} - \cos{\left(

In [30]:
eom[1]

1.0*I_r*Derivative(theta(t), (t, 2)) - 1.0*M_r*d*g*sin(theta(t)) - 1.0*g*h*m_b*sin(theta(t)) - 1.0*g*m_b*r*sin(theta(t)) - 1.0*g*m_b*x(t)*cos(theta(t)) + 1.0*h**2*m_b*Derivative(theta(t), (t, 2)) + 2.0*h*m_b*r*Derivative(theta(t), (t, 2)) + 1.0*h*m_b*cos(theta(t))*Derivative(x_w(t), (t, 2)) + 1.0*h*m_b*Derivative(x(t), (t, 2)) + 1.0*m_b*r**2*Derivative(theta(t), (t, 2)) + 1.0*m_b*r*cos(theta(t))*Derivative(x_w(t), (t, 2)) + 1.0*m_b*r*Derivative(x(t), (t, 2)) + 1.0*m_b*x(t)**2*Derivative(theta(t), (t, 2)) - 1.0*m_b*x(t)*sin(theta(t))*Derivative(x_w(t), (t, 2)) + 2.0*m_b*x(t)*Derivative(theta(t), t)*Derivative(x(t), t)

In [31]:
sympy.latex(eom[1])

'1.0 I_{r} \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} - 1.0 M_{r} d g \\sin{\\left(\\theta{\\left(t \\right)} \\right)} - 1.0 g h m_{b} \\sin{\\left(\\theta{\\left(t \\right)} \\right)} - 1.0 g m_{b} r \\sin{\\left(\\theta{\\left(t \\right)} \\right)} - 1.0 g m_{b} x{\\left(t \\right)} \\cos{\\left(\\theta{\\left(t \\right)} \\right)} + 1.0 h^{2} m_{b} \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} + 2.0 h m_{b} r \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} + 1.0 h m_{b} \\cos{\\left(\\theta{\\left(t \\right)} \\right)} \\frac{d^{2}}{d t^{2}} x_{w}{\\left(t \\right)} + 1.0 h m_{b} \\frac{d^{2}}{d t^{2}} x{\\left(t \\right)} + 1.0 m_{b} r^{2} \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} + 1.0 m_{b} r \\cos{\\left(\\theta{\\left(t \\right)} \\right)} \\frac{d^{2}}{d t^{2}} x_{w}{\\left(t \\right)} + 1.0 m_{b} r \\frac{d^{2}}{d t^{2}} x{\\left(t \\right)} + 1.0 m_{b} x^{2}{\\left(t \\right)} \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} - 1.0 m_{b} x{\\left(t \\ri

In [32]:
sympy.latex(eom[2])


'\\frac{1.0 I_{b} \\frac{d^{2}}{d t^{2}} x{\\left(t \\right)}}{r^{2}} - 1.0 g m_{b} \\sin{\\left(\\theta{\\left(t \\right)} \\right)} + 1.0 h m_{b} \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} + 1.0 m_{b} r \\frac{d^{2}}{d t^{2}} \\theta{\\left(t \\right)} - 1.0 m_{b} x{\\left(t \\right)} \\left(\\frac{d}{d t} \\theta{\\left(t \\right)}\\right)^{2} + 1.0 m_{b} \\cos{\\left(\\theta{\\left(t \\right)} \\right)} \\frac{d^{2}}{d t^{2}} x_{w}{\\left(t \\right)} + 1.0 m_{b} \\frac{d^{2}}{d t^{2}} x{\\left(t \\right)}'

In [33]:
M1 = Matrix([[(I_b/(r**2))+m_b , m_b*(h+r)],[m_b*(h+r),I_r+m_b*((h+r)**2)]])
print(sympy.simplify(M1))
print(sympy.latex(M1))


Matrix([[I_b/r**2 + m_b, m_b*(h + r)], [m_b*(h + r), I_r + m_b*(h + r)**2]])
\left[\begin{matrix}\frac{I_{b}}{r^{2}} + m_{b} & m_{b} \left(h + r\right)\\m_{b} \left(h + r\right) & I_{r} + m_{b} \left(h + r\right)^{2}\end{matrix}\right]


In [34]:
M2 = Matrix([[0,(g*m_b)],[(m_b*g), (M_r*d*g)+(m_b*g*(h+r))]])
print(sympy.simplify(M2))
print(sympy.latex(M2))

Matrix([[0, g*m_b], [g*m_b, g*(M_r*d + m_b*(h + r))]])
\left[\begin{matrix}0 & g m_{b}\\g m_{b} & M_{r} d g + g m_{b} \left(h + r\right)\end{matrix}\right]


In [35]:
M3 = Matrix([[-m_b*(h+r)],[-m_b*(h+r)**2 -I_r]])
print(sympy.simplify(M3))
print(sympy.latex(M3))

Matrix([[-m_b*(h + r)], [-I_r - m_b*(h + r)**2]])
\left[\begin{matrix}- m_{b} \left(h + r\right)\\- I_{r} - m_{b} \left(h + r\right)^{2}\end{matrix}\right]


In [ ]:
A1 = M1.inv()*M2
B1 = M1.inv()*M3
print(sympy.latex(sympy.simplify(A1)))

\left[\begin{matrix}- \frac{g m_{b}^{2} r^{2} \left(h + r\right)}{I_{b} I_{r} + I_{b} h^{2} m_{b} + 2 I_{b} h m_{b} r + I_{b} m_{b} r^{2} + I_{r} m_{b} r^{2}} & \frac{g m_{b} r^{2} \left(I_{r} - M_{r} d h - M_{r} d r\right)}{I_{b} I_{r} + I_{b} h^{2} m_{b} + 2 I_{b} h m_{b} r + I_{b} m_{b} r^{2} + I_{r} m_{b} r^{2}}\\\frac{g m_{b} \left(I_{b} + m_{b} r^{2}\right)}{I_{b} I_{r} + I_{b} h^{2} m_{b} + 2 I_{b} h m_{b} r + I_{b} m_{b} r^{2} + I_{r} m_{b} r^{2}} & \frac{g \left(I_{b} M_{r} d + I_{b} h m_{b} + I_{b} m_{b} r + M_{r} d m_{b} r^{2}\right)}{I_{b} I_{r} + I_{b} h^{2} m_{b} + 2 I_{b} h m_{b} r + I_{b} m_{b} r^{2} + I_{r} m_{b} r^{2}}\end{matrix}\right]


In [ ]:
print(sympy.latex(sympy.simplify(B1)))

\left[\begin{matrix}0\\-1\end{matrix}\right]
